This notebook tests whether the functions to import QARTOD test values from the qc-lookup repo correctly.

In [1]:
import qartod_testing.data_processing as dp
import xarray as xr

In [2]:
# Set reference designator, data stream, and method 

method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream
refdes = "CP01CNSM-RID27-03-CTDBPC000"               # build reference designator

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)


type = 'prod'                                       # dataset saved from OOINet/"production" 

In [3]:
ds_path = dp.build_data_path(refdes,method,stream,'prod')
ds_path

'..\\data\\interim\\prod-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered.nc'

In [4]:
# Load data from .nc files
ds = xr.open_dataset(ds_path)
ds

<xarray.Dataset>
Dimensions:                                            (time: 3380605)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) float32 ...
    sea_water_pressure                                 (time) float64 ...
    sea_water_electrical_conductivity_qartod_results   (time) float32 ...
    sea_water_pressure_qc_executed                     (time) float32 ...
    ctd_time                                           (time) datetime64[ns] ...
    sea_water_practical_salinity_qc_executed           (time) float32 ...
    ...                                                 ...
    sea_water_density                                  (time) float64 ...
    depth                                              (time) float64 ...
    sea_water_pressure_qartod_results                  (time) float32 ...
    sea_water_electrical_conductivity_qartod_executed  (time) |S64 ...
    sea_water_temperature                              (time) float64 ...
    sea_water_practical_salinity_qartod_executed       (time) |S64 ...
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13678
    lon:                                -70.76978

In [5]:
# Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
test_parameters={}
for var in ds.variables:
    if "qartod_results" in var:
        # Get the parameter name
        param = var.split("_qartod")[0]
        
        # Check if the parameter has an alternative ooinet_name
        if "alternate_parameter_name" in ds[param].attrs:
            ooinet_name = ds[param].attrs["alternate_parameter_name"]
        else:
            ooinet_name = param
        
        # Save the results in a dictionary
        test_parameters.update({
            param: ooinet_name
        })
# Print out the results
test_parameters

{'sea_water_electrical_conductivity': 'ctdbp_seawater_conductivity',
 'sea_water_temperature': 'ctdbp_seawater_temperature',
 'sea_water_practical_salinity': 'practical_salinity',
 'sea_water_pressure': 'ctdbp_seawater_pressure'}

In [6]:
# Example: load the gross range QARTOD table for a specific parameter
gross_range_qartod_test_values = dp.load_gross_range_qartod_test_values(refdes, stream, test_parameters["sea_water_temperature"])
gross_range_qartod_test_values

,subsite,node,sensor,stream,parameters,qcConfig,source,notes
200,CP01CNSM,RID27,03-CTDBPC000,ctdbp_cdef_instrument_recovered,{'inp': 'ctdbp_seawater_temperature'},{'qartod': {'gross_range_test': {'suspect_span...,Sensor min/max derived from vendor documentati...,NaN


In [7]:
# Example: load the climatology QARTOD table for a specific parameter
climatology_qartod_test_values = dp.load_climatology_qartod_test_values(refdes, test_parameters["sea_water_temperature"])
climatology_qartod_test_values

,"[1, 1]","[2, 2]","[3, 3]","[4, 4]","[5, 5]","[6, 6]","[7, 7]","[8, 8]","[9, 9]","[10, 10]","[11, 11]","[12, 12]"
"[0, 0]","[5.9399, 12.8153]","[6.1629, 9.5761]","[2.5334, 11.6971]","[4.136, 11.2716]","[4.8812, 15.9418]","[8.9832, 21.1365]","[12.3517, 27.6815]","[17.8339, 28.2053]","[17.9097, 27.5242]","[13.7813, 25.372]","[8.1263, 22.622]","[8.6744, 14.8909]"
